<a href="https://colab.research.google.com/github/OmNai12/CSCI---266---Adv-Database-System/blob/main/2_Sql_w_NEON_and_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install SQLAlchemy psycopg2-binary pandas

In [8]:
import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

NEON_DB_URL = os.environ.get("NEON_DB_URL") or getpass.getpass("Paste NEON DB URL (hidden): ")
engine = create_engine(NEON_DB_URL, pool_pre_ping=True, future=True)

Paste NEON DB URL (hidden): ··········


In [10]:
with engine.connect() as conn:
    q1 = """
    SELECT p.maker, pc.model, pc.price
    FROM product p
    JOIN pc ON p.model = pc.model
    WHERE pc.price = (
        SELECT MAX(pc2.price)
        FROM product p2
        JOIN pc pc2 ON p2.model = pc2.model
        WHERE p2.maker = p.maker
    )
    ORDER BY pc.price DESC;
    """
    q1_df = pd.read_sql_query(text(q1), conn)
q1_df


,maker,model,price
0,A,1001,2114
1,B,1006,1049
2,E,1011,959
3,D,1008,770
4,D,1010,770
5,C,1007,510


In [15]:
with engine.connect() as conn:
    q2 = """
    -- Query 2: Count of laptop models per maker
    SELECT p.maker, COUNT(*) AS laptop_count
    FROM product p
    JOIN laptop l ON p.model = l.model
    GROUP BY p.maker
    ORDER BY laptop_count DESC, p.maker;
    """
    q2_df = pd.read_sql_query(text(q2), conn)
q2_df



,maker,laptop_count
0,A,3
1,E,3
2,F,2
3,B,1
4,G,1


In [13]:
with engine.connect() as conn:
    q3 = """
    WITH all_products AS (
        SELECT p.maker, p.model, 'PC' AS type, pc.price
        FROM product p JOIN pc ON p.model = pc.model
        UNION ALL
        SELECT p.maker, p.model, 'Laptop', l.price
        FROM product p JOIN laptop l ON p.model = l.model
        UNION ALL
        SELECT p.maker, p.model, 'Printer', pr.price
        FROM product p JOIN printer pr ON p.model = pr.model
    )
    SELECT maker, type,
           MIN(price) AS min_price,
           MAX(price) AS max_price,
           ROUND(AVG(price),2) AS avg_price
    FROM all_products
    GROUP BY maker, type
    ORDER BY maker, type;
    """
    q3_df = pd.read_sql_query(text(q3), conn)
q3_df



,maker,type,min_price,max_price,avg_price
0,A,Laptop,1150,2500,1783.33
1,A,PC,478,2114,1195.67
2,B,Laptop,1429,1429,1429.00
3,B,PC,630,1049,776.00
4,C,PC,510,510,510.00
5,D,PC,650,770,730.00
6,D,Printer,120,120,120.00
7,E,Laptop,549,3673,1723.67
8,E,PC,529,959,712.33
9,E,Printer,99,899,412.33
